# CLV based RFM Analysis

---

**Outline**

1. Business Understanding
2. Modelling Workflow
3. Data Pipelines
4. EDA
5. Preprocessing
6. Clustering
7. Business Recommendation

## Import Package

In [1]:
import numpy as np
import pandas as pd 

import matplotlib.pyplot as plt
import seaborn as sns

# Case 1: Sales take place once a year

In [2]:
def calculate_clv(gross_contribution,
                  retention_rate,
                  discount_rate,
                  num_years,
                  promotional_expense):
    """
    Calculate the Customer Lifetime Value (CLV) for each customer.

    Parameters:
    gross_contribution: float
        The gross contribution for each customer
    
    retention_rate : float
        The yearly retention rate
    
    discount_rate : float
        The discount rate for marketing activities
    
    num_years : int
        The period of cash flows projection
    
    promotional_expenses: float
        The promotional expenses per customer per year

    Returns:
    --------
    clv : float
        The calculated CLV for the customer.
    """
    # Initiliaze the CLV for the current customer to 0
    clv = 0

    # Loop over the number of years
    for i in range(num_years):
        # Add the discounted gross contribution for year i to the clv
        clv += gross_contribution * (retention_rate ** i) \
               / ((1 + discount_rate) ** (i - 0.5))

        # Substract the discounted promotional expenses for year i from the CLV
        # (skip the 1st year because promotional expense start form 2nd year)
        if i > 0:
            clv -= promotional_expense * (retention_rate ** (i - 1)) \
                    / ((1 + discount_rate) ** (i - 0.5))
    
    # Return the calculated CLV
    return clv

In [5]:
clv = calculate_clv(gross_contribution= 250,
                    retention_rate=0.75,
                    discount_rate=0.2,
                    num_years=10,
                    promotional_expense=50)

In [7]:
print(f'CLV for customer with constant gross contribution'
      f'over 10 years is {np.round(clv,2)}')

CLV for customer with constant gross contributionover 10 years is 603.71


# Case 2: We Sales occur more frequently than once a year

In [ ]:
def calculate_clv_2(gross_contribution,
                    retention_rate,
                    discount_rate,
                    num_years,
                    num_cycles_per_year,
                    promotional_expense)
    """
    Calculate the Customer Lifetime Value (CLV) for each customer.

    Paramenters:
    ------------
    gross_contribution: float
        The gross contribution for each customer

    retention_rate: float
        The retention rate per sales cycle
    
    discount_rate: float
        The yearly discount rate for marketing activities

    num_years: int
        The period of cash flow projection in years

    num_cycles_per_year: int
        The number of sales cycles per year

    promotional_expenses: float
        The promotional expenses per customer per sales cycle

    Returns:
    -----------
    clv: float
        The calculated CLV for the customers.
    """
    # Calculate the adjusted retention rate and discount rate per cycle
    retention_rate_per_cycle = retention_rate ** (1/num_cycles_per_year)

    # Initialize the CLV for the current customer to 0
    clv = 0

    # Loop over the number of years
    for i in range(num_years*num_cycles_per_year):
        # ADd the discounted gross contribution for year i to the CLV
        clv += gross_contribution * (retention_rate ** i) \
            / ((1 + discount_rate) ** (i/num_cycles_per_year))
        
        # Subtract the discounted promotional expenses for year i from the CLV
        # (skip the 1st year because promotional expenses start from the 2nd year)
        if i > 0:
            clv -= promotional_expense * (retention_rate ** (i - 1)) \
                / ((1 + discount_rate) ** ((i - 0.5)/num_cycles_per_year))
        
    # Return the calculated CLV
    return clv